# EXAMPLE CROP GEOMETRY

In [29]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
import trimesh 

import os.path, time
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import operator
import copy

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [30]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
%autoreload 2

## 1. INPUTS

In [32]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath,"Samples11") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon

threshold=0.1

## 2. READ Data

In [33]:
# bimMeshesPaths=[os.path.join(sessionPath,"Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095339.ply") ,
#                          os.path.join(sessionPath,"Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095304.ply") ,
#                          os.path.join(sessionPath,"Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095314.ply") ,
#                          os.path.join(sessionPath,"Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095319.ply") ,
#                          os.path.join(sessionPath,"Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095324.ply") ,
#                          os.path.join(sessionPath,"Basic Wall_211_WA_Ff1_Glued brickwork sandlime 150mm_1095329.ply")]
# cutterMeshes=[o3d.io.read_triangle_mesh(path) for path in bimMeshesPaths]
# sourcePcd=o3d.io.read_point_cloud(os.path.join(sessionPath,'academiestraat week 22 19.pcd'))
# sourceMesh=o3d.io.read_triangle_mesh(os.path.join(sessionPath,'week22.obj'))
# cutterPcd=[sourcePcd.select_by_index([0,1,2,3]),sourcePcd.select_by_index([1000,1001,1002,1003])]


In [34]:
meshPath=os.path.join(sessionPath,"week22.obj")  
mesh= o3d.io.read_triangle_mesh(meshPath)

In [35]:
pcdPath=os.path.join(sessionPath,"academiestraat week 22 19.pcd")  
pcd= o3d.io.read_point_cloud(pcdPath) 

In [36]:
path=sessionPath
ifcPath2=os.path.join(path,"ifc_parking.ifc") 
classes= '.IfcBeam | .IfcColumn | .IfcWall | .IfcSlab'
ifc2 = ifcopenshell.open(ifcPath2)   

ifcSlab=ifc2.by_guid('2qZtnImXH6Tgdb58DjNlmF')
ifcWall=ifc2.by_guid('06v1k9ENv8DhGMCvKUuLQV')
ifcBeam=ifc2.by_guid('05Is7PfoXBjhBcbRTnzewz' )
ifcColumn=ifc2.by_guid('23JN72MijBOfF91SkLzf3a')
# ifcWindow=ifc.by_guid(cls.slabGlobalid) 
# ifcDoor=ifc.by_guid(cls.slabGlobalid)

slabMesh=gmu.ifc_to_mesh(ifcSlab)
wallMesh=gmu.ifc_to_mesh(ifcWall)
beamMesh=gmu.ifc_to_mesh(ifcBeam)
columnMesh=gmu.ifc_to_mesh(ifcColumn)

In [37]:
o3d.visualization.draw_geometries( [slabMesh,wallMesh,columnMesh,beamMesh] + [mesh]  )

In [38]:
box=mesh.get_axis_aligned_bounding_box()
boxPoints=np.asarray(box.get_box_points())
print(boxPoints)
cartesianBounds=gmu.get_cartesian_bounds(mesh)
print(cartesianBounds)
boundingPoints=np.asarray(gmu.get_bounding_points(cartesianBounds)) 
print(boundingPoints)

for i in range(0,7):
    for j in range(0,2):
        print(boundingPoints[i][j]-boxPoints[i][j])
        # self.assertAlmostEqual(boundingPoints[i][j],boxPoints[i][j],delta=0.01)

[[-37.36532974  16.87863541   0.71651864]
 [106.94235229  16.87863541   0.71651864]
 [-37.36532974 130.69406128   0.71651864]
 [-37.36532974  16.87863541  23.73304558]
 [106.94235229 130.69406128  23.73304558]
 [-37.36532974 130.69406128  23.73304558]
 [106.94235229  16.87863541  23.73304558]
 [106.94235229 130.69406128   0.71651864]]
[-37.36532974 106.94235229  16.87863541 130.69406128   0.71651864
  23.73304558]
[[-37.36532974  16.87863541   0.71651864]
 [106.94235229  16.87863541   0.71651864]
 [-37.36532974 130.69406128   0.71651864]
 [-37.36532974  16.87863541  23.73304558]
 [106.94235229 130.69406128  23.73304558]
 [-37.36532974 130.69406128  23.73304558]
 [106.94235229  16.87863541  23.73304558]
 [106.94235229 130.69406128   0.71651864]]
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


## crop_geometry_by_distance

In [23]:
test1=gmu.crop_geometry_by_distance(source=mesh,cutters=meshes)
print(test1)

test2=gmu.crop_geometry_by_distance(source=pcd,cutters=meshes)
print(test2)

test3=gmu.crop_geometry_by_distance(source=mesh,cutters=pcd)
print(test3)

test4=gmu.crop_geometry_by_distance(source=pcd,cutters=mesh)
print(test4)

[Open3D WARNING] [SelectByIndex] This mesh contains triangle uvs that are not handled in this function
TriangleMesh with 2024 points and 2471 triangles.
PointCloud with 361075 points.
[Open3D WARNING] [SelectByIndex] This mesh contains triangle uvs that are not handled in this function
TriangleMesh with 8596 points and 10387 triangles.
PointCloud with 1322636 points.


## crop_geometry_by_convex_hull

In [21]:
# cutters= [gmu.mesh_to_trimesh(mesh) for mesh in cutterMeshes]
sourceMesh=gmu.mesh_to_trimesh(mesh)
meshes=[slabMesh,wallMesh,columnMesh,beamMesh]
cutters=[gmu.mesh_to_trimesh(mesh) for mesh in meshes]

innerCrop=gmu.crop_geometry_by_convex_hull(source=sourceMesh, cutters=cutters, inside = True )
innerCrop=[mesh.as_open3d for mesh in innerCrop]
print(innerCrop)


unable to convert colors!


[TriangleMesh with 12563 points and 10262 triangles., TriangleMesh with 814 points and 669 triangles., TriangleMesh with 1036 points and 893 triangles., TriangleMesh with 1266 points and 1169 triangles.]


In [12]:
boxes=[mesh.get_oriented_bounding_box() for mesh in innerCrop]
for box in boxes:
    box.color=[1,0,0]
innerCrop=[mesh.paint_uniform_color([1, 0.706, 0]) for mesh in innerCrop]
# hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
# hull_ls.paint_uniform_color((1, 0, 0))
# o3d.visualization.draw_geometries( meshes +test3 + [hull_ls]  )
o3d.visualization.draw_geometries( [mesh] + boxes + innerCrop  )


In [14]:
outerCrop=gmu.crop_geometry_by_convex_hull(source=sourceMesh, cutters=cutters[0], inside = False ) 
outerCrop=[mesh.as_open3d for mesh in outerCrop]
print(outerCrop)
# hull, _ = sourcePcd.compute_convex_hull()

[TriangleMesh with 255157 points and 499966 triangles.]


In [19]:
boxes=[mesh.get_oriented_bounding_box() for mesh in outerCrop]
for box in boxes:
    box.color=[1,0,0]
outerCrop=[mesh.paint_uniform_color([1, 0.706, 0]) for mesh in outerCrop]
# hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
# hull_ls.paint_uniform_color((1, 0, 0))
# o3d.visualization.draw_geometries( meshes +test3 + [hull_ls]  )
o3d.visualization.draw_geometries( boxes + outerCrop + [slabMesh] )

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


## crop_geometry_by_raycasting

In [81]:
pcd=sourcePcd
mesh=sourceMesh
hull, _ = pcd.compute_convex_hull()

test1=gmu.crop_geometry_by_raycasting(source=mesh, cutter=hull, inside=True, strict =True ) 
print(test1)

test2=gmu.crop_geometry_by_raycasting(source=mesh, cutter=hull, inside=True, strict =False ) 
print(test2)

test3=gmu.crop_geometry_by_raycasting(source=mesh, cutter=hull, inside=False, strict =True ) 
print(test3)

[Open3D WARNING] [RemoveDegenerateTriangles] This mesh contains triangle uvs that are not handled in this function
TriangleMesh with 173154 points and 243839 triangles.
TriangleMesh with 174586 points and 246108 triangles.
[Open3D WARNING] [RemoveDegenerateTriangles] This mesh contains triangle uvs that are not handled in this function
TriangleMesh with 157109 points and 236652 triangles.


In [90]:
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([test3] + [hull_ls]  )